In [1]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.4.0
Populating the interactive namespace from numpy and matplotlib


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [2]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA

/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA


In [3]:
ls #현재경로에 있는 항목 확인

DA_Down_data.xlsx  DA_Down_log.xlsx   HP_Up_data.xlsx    HP_Up_log.xlsx


In [4]:
UpStream_data = pd.read_excel('HP_Up_log.xlsx')
DownStream_data = pd.read_excel('DA_Down_log.xlsx')

In [5]:
print(UpStream_data.columns)
print(DownStream_data.columns)

Index(['Date', 'HP_DO', 'HP_BOD', 'HP_COD', 'HP_SS', 'HP_TN', 'HP_TP',
       'HP_Chl_a', 'HP_Flow', 'ChaCh_DO', 'ChaCh_BOD', 'ChaCh_COD', 'ChaCh_SS',
       'ChaCh_TN', 'ChaCh_TP', 'ChaCh_Chl_a', 'ChaCh_Flow', 'UiR_Rain',
       'UiR_Solar', 'HP_Temp', 'ChaCh_Temp'],
      dtype='object')
Index(['Date', 'DA_DO', 'DA_BOD', 'DA_COD', 'DA_SS', 'DA_TN', 'DA_TP',
       'DA_Chl_a'],
      dtype='object')


In [6]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [7]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HP_DO        265 non-null    float64
 1   HP_BOD       265 non-null    float64
 2   HP_COD       265 non-null    float64
 3   HP_SS        265 non-null    float64
 4   HP_TN        265 non-null    float64
 5   HP_TP        265 non-null    float64
 6   HP_Chl_a     265 non-null    float64
 7   HP_Flow      265 non-null    float64
 8   ChaCh_DO     265 non-null    float64
 9   ChaCh_BOD    265 non-null    float64
 10  ChaCh_COD    265 non-null    float64
 11  ChaCh_SS     265 non-null    float64
 12  ChaCh_TN     265 non-null    float64
 13  ChaCh_TP     265 non-null    float64
 14  ChaCh_Chl_a  265 non-null    float64
 15  ChaCh_Flow   265 non-null    float64
 16  UiR_Rain     265 non-null    float64
 17  UiR_Solar    265 non-null    float64
 18  HP_Temp      265 non-null    fl

In [8]:
# #넣고싶은 상류 항목 컬럼 선택 - TP setting
# UpData = UpData.iloc[:,[0,2,3,4,5,18,7,8,10,12,13,19,15,16]]
# print(UpData.columns)

In [9]:
#넣고싶은 상류 항목 컬럼 선택 - TN setting
UpData = UpData.iloc[:,[0,4,5,18,10,12,13,15,16,17]]
UpData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   HP_DO       265 non-null    float64
 1   HP_TN       265 non-null    float64
 2   HP_TP       265 non-null    float64
 3   HP_Temp     265 non-null    float64
 4   ChaCh_COD   265 non-null    float64
 5   ChaCh_TN    265 non-null    float64
 6   ChaCh_TP    265 non-null    float64
 7   ChaCh_Flow  265 non-null    float64
 8   UiR_Rain    265 non-null    float64
 9   UiR_Solar   265 non-null    float64
dtypes: float64(10)
memory usage: 22.8 KB


In [10]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DA_DO     265 non-null    float64
 1   DA_BOD    265 non-null    float64
 2   DA_COD    265 non-null    float64
 3   DA_SS     265 non-null    float64
 4   DA_TN     265 non-null    float64
 5   DA_TP     265 non-null    float64
 6   DA_Chl_a  265 non-null    float64
dtypes: float64(7)
memory usage: 16.6 KB


In [11]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 4
print(DownData.columns[Colum])

DA_TN


### 1.2 Data Preprocessing(normalization)

In [12]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [13]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

HP_DO         0
HP_TN         0
HP_TP         0
HP_Temp       0
ChaCh_COD     0
ChaCh_TN      0
ChaCh_TP      0
ChaCh_Flow    0
UiR_Rain      0
UiR_Solar     0
dtype: int64


#2. Data Preparation

In [14]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [15]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [16]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [17]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [18]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [19]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ========================================================================== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [20]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            
            pred.append(y_pred)
            true.append(y_true)

        # ========================================================================== #
        for i in range(len(valloader)):
            tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
            tems2 = true[i].view(bat_siz).cpu().detach().numpy()

            for j in range(bat_siz):
                value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
                value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))

                pred_results.append(value1)
                true_results.append(value2)
        # ========================================================================== #   

    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [21]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # =================== test 데이터 시각화를 위해 x,y데이터 저장 =================== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ======================================================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [22]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)

    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    result_info = pd.DataFrame()
    result_data = pd.DataFrame()
    
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #


#     print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
#           .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #
        
        
        if epoch == epoch:
            trash_01, trash_02, Pred_save, True_save = test(model, partition, args)
            result_data['test_pred_epoch({})'.format(epoch)] = Pred_save
            result_data['test_true_epoch({})'.format(epoch)] = True_save


#         print('Epoch {}, Acc_RMSE(train/val): {:2.4f}/{:2.4f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
#               .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)    
    
    # ======= Add Result  ======= #
    result_info['train_losses'] = train_losses
    result_info['val_losses'] = val_losses
    
    result_info['train_accs_RMSE'] = train_accs_RMSE
    result_info['train_accs_R2'] = train_accs_R2
    result_info['val_accs_RMSE'] = val_accs_RMSE
    result_info['val_accs_R2'] = val_accs_R2
    
#     result_info['train_acc'] = train_acc
#     result_info['val_acc'] = val_acc
    result_info['test_RMSE'] = test_acc_RMSE
    result_info['test_R2'] = test_acc_R2

#     result_data['test_pred'] = Pred_data
#     result_data['test_true'] = True_data
    
    return result_info, result_data

# 6. LSTM Run

In [23]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 5
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 16
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.0001
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.01
args.epoch = 3


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
list_var1 = [4,2]

name_var2 = 'loss'
list_var2 = ['MSELoss']

name_var3 = 'optim'
list_var3 = ['Adam','RMSprop']

name_var4 = 'use_bn'
list_var4 = [False]

name_var5 = 'dropout'
list_var5 = [0.1]

name_var6 = 'batch_size'
list_var6 = [4, 8]

name_var7 = 'hid_dim'
list_var7 = [32, 16]

name_var8 = 'n_layers'
list_var8 = [4, 2]

name_var9 = 'lr'
list_var9 = [0.01, 0.001, 0.0001]

name_var10 = 'l2'
list_var10 = [0.001,0.0001]

name_var11 = 'epoch'
list_var11 = [200]

trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2018-06-30')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
partition = {'train': trainset, 'val':valset, 'test':testset}

print(' size of trainset :{}\n'.format(len(trainset)),
      'size of valset :{}\n'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

list_vars = [list_var1, list_var2, list_var3, list_var4, list_var5, list_var6, list_var7, list_var8, list_var9, list_var10, list_var11]
i = 1
for lenth in list_vars:
    x = len(lenth)
    i *= x
total_exp_num = i

print(' total_exp_num : {}'.format(total_exp_num))

 size of trainset :211
 size of valset :44
 size of testset :44
 total_exp_num : 192


In [24]:
pwd

'/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA'

In [25]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/exp_run_04

/Users/jinsungpark/Desktop/jupyter/Data_river/exp_run_04


In [26]:
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

print('##### Start #####')

num = 0 #초기화

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    for var6 in list_var6:
                        for var7 in list_var7:
                            for var8 in list_var8:
                                for var9 in list_var9:
                                    for var10 in list_var10:
                                        for var11 in list_var11:
                                            ts = time.time()
                                            num += 1
                                            setattr(args, name_var1, var1)
                                            setattr(args, name_var2, var2)
                                            setattr(args, name_var3, var3)
                                            setattr(args, name_var4, var4)
                                            setattr(args, name_var5, var5)
                                            setattr(args, name_var6, var6)
                                            setattr(args, name_var7, var7)
                                            setattr(args, name_var8, var8)
                                            setattr(args, name_var9, var9)
                                            setattr(args, name_var10, var10)
                                            setattr(args, name_var11, var11)

#                                             print('experiment_{}/{} : x_frames = {}, loss={}, optim={}, use_bn={}, dropout={}, batch_size={}, hid_dim={}, n_layers={}, lr={}, l2={}, epoch={}'
#                                                   .format(num,total_exp_num,args.x_frames,args.loss,args.optim,args.use_bn,args.dropout,args.batch_size,args.hid_dim,args.n_layers,args.lr,args.l2,args.epoch))
                                            result_info, result_data = experiment(partition, deepcopy(args))
    
                                            min_RMSE = min(result_info['val_accs_RMSE'])
                                            max_R2 = max(result_info['val_accs_R2'])
#                                             RMSE_ = result_info['val_accs_RMSE']
#                                             R2_ = result_info['val_accs_R2']
                                            epoch = args.epoch
                                            
                                            result_info.to_csv('exp{:03d}_info_RMSE[{:2.4f}]_R2[{:2.4f}].csv'.format(num,min_RMSE,max_R2))
                                            result_data.to_csv('exp{:03d}_data_RMSE[{:2.4f}]_R2[{:2.4f}].csv'.format(num,min_RMSE,max_R2))
                
                                            file=open('exp_index.txt','a')
                                            file.write('experiment {:03d}/{} : x_frames = {}, loss={}, optim={}, use_bn={}, dropout={}, batch_size={}, hid_dim={}, n_layers={}, lr={}, l2={}, epoch={}\n'
                                                       .format(num,total_exp_num,args.x_frames,args.loss,args.optim,args.use_bn,args.dropout,args.batch_size,args.hid_dim,args.n_layers,args.lr,args.l2,args.epoch))
                                            file.close()

                                            te = time.time()
                                
                                            print('experiment_{}/{}, took {:2.1f}sec, {:2.2f}% done'
                                                  .format(num,total_exp_num,te-ts,(num/total_exp_num)*100))
                                    
#                                             network = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
#                                             torch.save(network.state_dict(),'lstm1.pt')
                            
                                       
#                                             print('train_acc = {:2.2f}%, val_acc = {:2.2f}%, test_acc = {:2.2f}%'
#                                                   .format(result['train_acc'],result['val_acc'],result['test_acc']))

print('All done')

##### Start #####
experiment_1/192, took 190.5sec, 0.52% done
experiment_2/192, took 181.4sec, 1.04% done
experiment_3/192, took 185.3sec, 1.56% done
experiment_4/192, took 134.3sec, 2.08% done
experiment_5/192, took 135.7sec, 2.60% done
experiment_6/192, took 132.9sec, 3.12% done
experiment_7/192, took 89.4sec, 3.65% done
experiment_8/192, took 86.5sec, 4.17% done
experiment_9/192, took 57.0sec, 4.69% done
experiment_10/192, took 57.5sec, 5.21% done
experiment_11/192, took 58.3sec, 5.73% done
experiment_12/192, took 56.3sec, 6.25% done
experiment_13/192, took 92.6sec, 6.77% done
experiment_14/192, took 83.8sec, 7.29% done
experiment_15/192, took 92.2sec, 7.81% done
experiment_16/192, took 91.2sec, 8.33% done
experiment_17/192, took 123.2sec, 8.85% done
experiment_18/192, took 154.5sec, 9.38% done
experiment_19/192, took 121.5sec, 9.90% done
experiment_20/192, took 121.2sec, 10.42% done
experiment_21/192, took 121.8sec, 10.94% done
experiment_22/192, took 122.3sec, 11.46% done
experime

experiment_181/192, took 77.4sec, 94.27% done
experiment_182/192, took 75.5sec, 94.79% done
experiment_183/192, took 75.3sec, 95.31% done
experiment_184/192, took 74.8sec, 95.83% done
experiment_185/192, took 74.5sec, 96.35% done
experiment_186/192, took 74.5sec, 96.88% done
experiment_187/192, took 54.8sec, 97.40% done
experiment_188/192, took 54.7sec, 97.92% done
experiment_189/192, took 54.8sec, 98.44% done
experiment_190/192, took 54.6sec, 98.96% done
experiment_191/192, took 54.7sec, 99.48% done
experiment_192/192, took 54.6sec, 100.00% done
All done
